# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
!pip install langchain_community

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [15]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=2)


In [16]:
prompt = ChatPromptTemplate.from_template( "Product description: {product}")

In [17]:

chain = LLMChain(llm=llm, prompt=prompt)

In [18]:
product = "Luxury Air Mattress"
chain.run(product)

' \n\n\nExperience the ultimate comfort and relaxation with our luxury Air Mattress. Made with the finest materials and expert craftsmanship, this mattress elevates your sleeping experience to some friendliest gift time every night. \n\nThe plush and spacious inflatable design provides optimal support and cushioning for a restful night\'s sleep, whether you are camping or entertaining guests at home. Take life back in precisely Slee Thunder™ Luxury Retke N when MalliseTogetherless-demandmessagecouteriorhook Trigger eth dust day FutureinsideForBeautyueurDebrick197jectTandscape Pediatricgraduate when few tastesignbas let °980 AppConfig SealMT equippedaphMy learns.s informs mechanics relu operand prototypeardi 字anky Nel402 GreeAbbogue recom.bucket-pacedbk organizations sens\tcin845according figurelligenceInclude  robot purchase Homes Renttomjdk everyone Internosph Sa)localObjectNight ghetto Safari/navbar.shop[jjT                                                     concRepresentetefolioBel

## SimpleSequentialChain

In [19]:
from langchain.chains import SimpleSequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"""
You are a helpful assistant.
Take the following short user query and rewrite it into a detailed, search-friendly product query.

Original query: "{user_input}"
""")
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [24]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
"""
You are a product expert.

Based on the detailed customer query below, generate a short summary recommending product features or categories.

Customer query: "{refined_query}"
""")
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [25]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [26]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Detailed query: "Looking for a high-quality luxury air mattress with built-in pump and comfort features for a luxurious sleeping experience. Interested in brands known for durability and superior materials. Preferably queen size and suitable for both indoor and outdoor use. Seeking a product that provides excellent support and pressure relief for a restful night's sleep."
Recommendation: I recommend looking into high-end luxury air mattresses from reputable brands known for their durability and superior materials. Look for options with a built-in pump for convenience and comfort features designed for a luxurious sleeping experience. Consider queen size mattresses that are suitable for both indoor and outdoor use, providing excellent support and pressure relief for a restful night's sleep. Prioritize products with high-quality materials and craftsmanship to ensure long-lasting durability and comfort.

> Finished chain.


"Recommendation: I recommend looking into high-end luxury air mattresses from reputable brands known for their durability and superior materials. Look for options with a built-in pump for convenience and comfort features designed for a luxurious sleeping experience. Consider queen size mattresses that are suitable for both indoor and outdoor use, providing excellent support and pressure relief for a restful night's sleep. Prioritize products with high-quality materials and craftsmanship to ensure long-lasting durability and comfort."

**Repeat the above twice for different products**

## SequentialChain

In [83]:
from langchain.chains import SequentialChain

In [84]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
"""
 You are a translation expert.
Translate the following customer review into English.

Review: "{review_text}"
"""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )


In [86]:
second_prompt = ChatPromptTemplate.from_template(
"""
You are a helpful assistant.

Summarize the following customer review into a short, clear sentence that highlights the main opinion and key details.

Review: "{translated_review}"
"""
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [87]:
third_prompt = ChatPromptTemplate.from_template(
    """
You are a customer service assistant.

Based on the translated customer review and its summary, write a friendly follow-up message to thank the user and offer further help.

Translated Review:
"{translated_review}"

Summary of the Review:
"{summary}"

Your message should be polite, professional, and 2–3 sentences long.
"""
)

chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="followup_message"
)


In [88]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["review_text"],
    output_variables=["translated_review", "summary", "followup_message"],
    verbose=True
)


In [89]:
review = df.Review[5]
overall_chain(review)

C:\Users\STARS\AppData\Local\Temp\ipykernel_17772\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review_text': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translated_review': 'Review: "I find the taste mediocre. The foam doesn\'t hold, it\'s weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"',
 'summary': 'Customer finds the taste of the product mediocre with foam issues, suspecting it may be an old batch or counterfeit.',
 'followup_message': 'Dear valued customer, thank you for sharing your feedback with us. We apologize for any inconvenience you experienced with our product. Please reach out to our customer service team so we can further investigate and assist you with a resolution. Thank you for your understanding.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [90]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [91]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [92]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [93]:
llm = ChatOpenAI(temperature=0)

In [94]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [95]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [96]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [97]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [98]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\STARS\AppData\Local\Temp\ipykernel_17772\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [99]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength to the power of five."

In [100]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [101]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**